In [1]:
import sys,os,os.path

sys.path.append("../../")   # cite IC from parent directory
                            # NOTE if you can't import IC stuff, its because of the
                            # above line
#sys.path.append(os.path.expanduser('~/code/eol_hsrl_python'))
os.environ['ICTDIR']='/home/e78368jw/Documents/NEXT_CODE/IC'

import matplotlib.pyplot as plt
import pandas as pd
import numpy  as np
import tables as tb
import IC.invisible_cities.io.dst_io                           as     dstio
import IC.invisible_cities.io.mcinfo_io as mcio
from    IC.invisible_cities.core.core_functions   import shift_to_bin_centers
import iminuit,probfit

import scipy.special as special
from scipy.stats import skewnorm
from scipy.optimize import curve_fit



In [4]:
from FOM_scripts import load_tracks

In [5]:
tracks = load_tracks('../LPR/208Tl/PORT_2a/prod/isaura/isaura/', filter = 1)

Warning! This method may take some time,
and works best with smaller datasets (like LPR).
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/e78368jw/anaconda3/envs/IC-3.8-2022-04-13/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_33780/3928216829.py", line 1, in <module>
    tracks = load_tracks('../LPR/208Tl/PORT_2a/prod/isaura/isaura/', filter = 1)
  File "/home/e78368jw/Documents/NEXT_CODE/Th_studies/isaura_study_FOM/FOM_scripts.py", line 88, in load_tracks
  File "/home/e78368jw/Documents/NEXT_CODE/Th_studies/isaura_study_FOM/../../IC/invisible_cities/io/dst_io.py", line 46, in load_dst
    return read_dst_(filename, group, node, evt_list)
  File "/home/e78368jw/Documents/NEXT_CODE/Th_studies/isaura_study_FOM/../../IC/invisible_cities/io/dst_io.py", line 28, in read_dst_
    table  = getattr(getattr(h5in.root, group), node)
  File "/home/e78368jw/anaconda3/envs/IC-3.8-2022-04-13/lib/python3.8/site-packages/tables/group.py", line 798, in __getattr__
    retur

In [10]:
display(tracks.tail())

,event,trackID,energy,length,numb_of_voxels,numb_of_hits,numb_of_tracks,x_min,y_min,z_min,...,blob1_z,blob2_x,blob2_y,blob2_z,eblob1,eblob2,ovlp_blob_energy,vox_size_x,vox_size_y,vox_size_z
22210,4060432,0,1.039139,38.755708,44,4508,1,-242.325,173.675,863.608704,...,865.874337,-221.907829,178.657348,886.930067,0.786380,0.614325,0.372762,9.20,10.000000,4.493522
22211,4060438,0,0.851373,100.307208,38,2347,1,-150.325,238.675,1071.537964,...,1091.459595,-132.592985,243.951636,1078.853394,0.034897,0.016382,0.000000,10.00,10.000000,4.980408
22212,4060446,0,0.452397,40.827108,32,2247,1,361.675,267.675,1020.069702,...,1038.269608,366.332921,303.210335,1021.775019,0.405295,0.041327,0.008941,9.75,10.000000,4.980133
22213,4060450,0,0.689823,30.717052,28,3174,1,362.675,14.675,1143.984375,...,1149.357396,369.935625,27.036415,1169.917556,0.609078,0.594648,0.513903,9.25,8.833333,4.893758
22214,4060460,0,0.585209,32.031497,31,2887,1,-110.325,132.675,1159.739380,...,1179.621641,-90.086359,137.842825,1160.199245,0.468832,0.394256,0.277879,8.00,8.500000,4.816992


In [3]:
from FOM_scripts import load_MC

In [5]:
test_data = load_MC('../LPR/208Tl/PORT_2a/prod/isaura/isaura/')

In [19]:
# removing eventmap info
test_data = test_data[1]

In [24]:
print(test_data.columns)
display(test_data)

Index(['event_id', 'particle_id', 'particle_name', 'primary', 'mother_id',
       'initial_x', 'initial_y', 'initial_z', 'initial_t', 'final_x',
       'final_y', 'final_z', 'final_t', 'initial_volume', 'final_volume',
       'initial_momentum_x', 'initial_momentum_y', 'initial_momentum_z',
       'final_momentum_x', 'final_momentum_y', 'final_momentum_z',
       'kin_energy', 'length', 'creator_proc', 'final_proc'],
      dtype='object')


,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
0,2540000,1,Tl208,True,0,394.379333,397.299896,1123.894287,0.000000,394.379333,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,none,RadioactiveDecay
1,2540000,4,e-,False,1,394.379333,397.299896,1123.894287,0.000329,397.574554,...,0.298550,-0.500215,-0.120595,0.000000,0.0000,0.000000,0.273227,6.134542,RadioactiveDecay,eIoni
2,2540000,3,anti_nu_e,False,1,394.379333,397.299896,1123.894287,0.000329,3386.472656,...,0.284348,-0.560600,1.081109,0.284348,-0.5606,1.081109,1.250569,13159.272461,RadioactiveDecay,Transportation
3,2540000,2,Pb208[3475.078],False,1,394.379333,397.299896,1123.894287,0.000329,394.379333,...,-0.582898,1.060815,-0.960514,-0.000000,0.0000,-0.000000,0.000006,0.000000,RadioactiveDecay,RadioactiveDecay
4,2540000,6,gamma,False,2,394.379333,397.299896,1123.894287,0.000329,322.789032,...,-0.678180,-0.522776,-0.085619,0.000000,-0.0000,0.000000,0.860554,1767.570312,RadioactiveDecay,phot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5724192,2030235,51,e-,False,33,-52.155415,-376.568848,922.736206,3.119910,-52.155907,...,-0.014988,0.017691,0.004070,-0.000000,0.0000,0.000000,0.000542,0.002932,compt,eIoni
5724193,2030235,50,e-,False,33,-52.155415,-376.568848,922.736206,3.119910,-50.344692,...,-0.318292,-0.518366,0.062670,-0.000000,0.0000,-0.000000,0.285908,37.882458,compt,eIoni
5724194,2030235,88,e-,False,50,-53.210651,-381.387848,923.548096,3.142658,-52.953606,...,0.008486,-0.228941,0.153380,-0.000000,-0.0000,-0.000000,0.069631,6.677827,eIoni,eIoni
5724195,2030235,30,e-,False,29,396.589569,390.954346,1142.969971,0.063078,396.598022,...,-0.320552,-0.079711,0.683712,-0.000000,-0.0000,0.000000,0.404255,0.242132,conv,eIoni


In [25]:
display(test_data[test_data['particle_name'] == 'e+'])

,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
442,2540006,28,e+,False,26,397.994476,406.192200,1104.511230,0.081588,397.663483,...,-0.113898,0.392981,-1.828005,0.0,0.0,-0.0,1.430683,1.431519,conv,annihil
779,2540009,36,e+,False,34,417.531311,434.714142,1134.706055,0.147440,417.572449,...,1.069797,0.628779,-0.066560,-0.0,0.0,-0.0,0.832645,0.483091,conv,annihil
1620,2540018,35,e+,False,33,351.407288,195.293655,1046.552856,0.731658,388.764038,...,-0.395765,-1.658938,-0.268138,-0.0,0.0,-0.0,1.289479,415.102295,conv,annihil
2175,2540023,39,e+,False,37,356.425354,-46.521172,775.359009,1.884820,364.295624,...,0.227442,-0.875990,-0.139887,0.0,-0.0,-0.0,0.537704,127.085846,conv,annihil
2370,2540025,73,e+,False,71,234.638184,449.706116,1324.644409,0.869903,227.466187,...,-0.485283,-0.023844,1.693891,-0.0,0.0,0.0,1.323792,9.668751,conv,annihil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5721026,2030198,55,e+,False,53,-317.644165,122.913124,780.973267,2.805562,-227.563492,...,-1.560910,-0.481398,-0.572206,0.0,0.0,0.0,1.293640,398.153534,conv,annihil
5721146,2030199,29,e+,False,27,388.389008,145.365875,1089.239014,0.848894,388.548676,...,-0.078389,-0.186970,0.119934,-0.0,-0.0,-0.0,0.051679,3.023956,conv,annihil
5722787,2030217,91,e+,False,87,210.304977,-88.572311,528.882324,2.638315,209.844193,...,0.204529,-0.242675,-0.472967,0.0,0.0,0.0,0.254208,50.648548,conv,annihil
5723616,2030228,34,e+,False,32,366.878693,391.280914,1094.120483,0.133110,365.349640,...,-0.834090,-0.260838,-0.929550,0.0,-0.0,-0.0,0.863383,4.327487,conv,annihil


In [8]:
space = np.linspace(0,1000, 1001, endpoint = True)
print(space)

[   0.    1.    2. ...  998.  999. 1000.]


In [12]:
# chunk testing
for i in range(len(space)):
    if ((i%np.floor(len(space)*0.1) == 0)):
        print("Chunk")
        print(i)


Chunk
0
Chunk
100
Chunk
200
Chunk
300
Chunk
400
Chunk
500
Chunk
600
Chunk
700
Chunk
800
Chunk
900
Chunk
1000


In [1]:
import FOM_scripts as FScripts

In [2]:
pos_data = FScripts.positron_scraper('../LPR/208Tl/PORT_2a/prod/isaura/isaura/', save = True)

Chunking at event 30!
914 positron events found
914 positron events total
Chunking at event 60!
975 positron events found
1889 positron events total
Chunking at event 90!
964 positron events found
2853 positron events total
Chunking at event 120!
967 positron events found
3820 positron events total
Chunking at event 150!
964 positron events found
4784 positron events total
Chunking at event 180!
936 positron events found
5720 positron events total
Chunking at event 210!
1022 positron events found
6742 positron events total
Chunking at event 240!
989 positron events found
7731 positron events total
Chunking at event 270!
943 positron events found
8674 positron events total
Chunking at event 300!
989 positron events found
9663 positron events total


/home/e78368jw/anaconda3/envs/IC-3.8-2022-04-13/lib/python3.8/site-packages/pandas/core/generic.py:2703: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block1_values] [items->Index(['event_id', 'particle_id', 'particle_name', 'primary', 'mother_id',
       'initial_volume', 'final_volume', 'creator_proc', 'final_proc'],
      dtype='object')]

  pytables.to_hdf(


In [5]:
import pandas as pd
pandas_test = pd.read_hdf('./positrons.h5')

In [6]:
display(pandas_test)

,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
442,2540006,28,e+,False,26,397.994476,406.192200,1104.511230,0.081588,397.663483,...,-0.113898,0.392981,-1.828005,0.0,0.0,-0.0,1.430683,1.431519,conv,annihil
779,2540009,36,e+,False,34,417.531311,434.714142,1134.706055,0.147440,417.572449,...,1.069797,0.628779,-0.066560,-0.0,0.0,-0.0,0.832645,0.483091,conv,annihil
1620,2540018,35,e+,False,33,351.407288,195.293655,1046.552856,0.731658,388.764038,...,-0.395765,-1.658938,-0.268138,-0.0,0.0,-0.0,1.289479,415.102295,conv,annihil
2175,2540023,39,e+,False,37,356.425354,-46.521172,775.359009,1.884820,364.295624,...,0.227442,-0.875990,-0.139887,0.0,-0.0,-0.0,0.537704,127.085846,conv,annihil
2370,2540025,73,e+,False,71,234.638184,449.706116,1324.644409,0.869903,227.466187,...,-0.485283,-0.023844,1.693891,-0.0,0.0,0.0,1.323792,9.668751,conv,annihil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571336,2030198,55,e+,False,53,-317.644165,122.913124,780.973267,2.805562,-227.563492,...,-1.560910,-0.481398,-0.572206,0.0,0.0,0.0,1.293640,398.153534,conv,annihil
571456,2030199,29,e+,False,27,388.389008,145.365875,1089.239014,0.848894,388.548676,...,-0.078389,-0.186970,0.119934,-0.0,-0.0,-0.0,0.051679,3.023956,conv,annihil
573097,2030217,91,e+,False,87,210.304977,-88.572311,528.882324,2.638315,209.844193,...,0.204529,-0.242675,-0.472967,0.0,0.0,0.0,0.254208,50.648548,conv,annihil
573926,2030228,34,e+,False,32,366.878693,391.280914,1094.120483,0.133110,365.349640,...,-0.834090,-0.260838,-0.929550,0.0,-0.0,-0.0,0.863383,4.327487,conv,annihil
